# Assignment

In [67]:
from bs4 import BeautifulSoup
import requests
from pandas.io.json import json_normalize 
import pandas as pd

## Request the html and Use BeautifulSoup to extract the content

In [8]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
results = requests.get(url)
soup = BeautifulSoup(results.content)

## Extract the table and its headings

In [34]:
post_code_list = soup.find("table", attrs={"class": "wikitable sortable"})
post_code_list_data = post_code_list.tbody.find_all("tr")  

# Get all the headings of the table
headings = []
for th in post_code_list_data[0].find_all("th"):
    # remove any newlines and extra spaces from left and right
    headings.append(th.text.replace('\n', ' ').strip())

print(headings)

['Postal Code', 'Borough', 'Neighbourhood']


## Generate the table on the web

In [116]:
data = []

for row in post_code_list_data[1:]:
    t_row = {}
    for td, th in zip(row.find_all('td'), headings):
        t_row[th] = td.text.replace('\n', '').strip()
    data.append(t_row)
post_table = pd.DataFrame(data)
post_table

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


## Drop the lines with Borough not assigned

In [95]:
new_table = post_table.loc[post_table['Borough']!='Not assigned',:]
new_table

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [117]:
# It looks there is no repeated Postal Code line in the table
len(pd.unique(new_table['Postal Code']))

103

## Assign the not-assigned neighborhood with the borough, clean table and generate the final table and it's shape 

In [119]:
for neighbor, i in enumerate(new_table['Neighbourhood']):
    if neighbor == 'Not assigned':
        new_table.loc[i,'Neighbourhood'] = new_table.loc[i,'Borough']
new_table.reset_index(drop=True, inplace=True)
print('The number of rows of my dataframe is',new_table.shape[0])
new_table

The number of rows of my dataframe is 103


,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."
